# Named Entity Recognition (NER) Assignment

In this assignment, you will fine-tune a pre-trained transformer model for a Named Entity Recognition (NER) task using the CoNLL-2003 dataset.

## Step 1: Setup
Let's start by installing the necessary libraries.

In [ ]:
# Install the transformers library
!pip install transformers datasets
!pip install transformers[torch]
!pip install seqeval

In [ ]:
# Install the necessary libraries
!pip install transformers datasets seqeval

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric, ClassLabel
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Step 2: Load Dataset
We will use the CoNLL-2003 dataset, which is available through the `datasets` library.

In [ ]:
# Load the CoNLL-2003 dataset
datasets = load_dataset("conll2003", trust_remote_code=True)
label_list = datasets["train"].features["ner_tags"].feature.names

# Display dataset structure
datasets

## Step 3: Tokenization
We need to tokenize the dataset using a pre-trained tokenizer.

In [ ]:
# Load a pre-trained tokenizer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize the dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding="max_length")
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])# append label of patricular word index in label_ids # begin code() # end code()
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Apply the tokenization and alignment function to the dataset
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets.set_format("torch")

In [ ]:
tokenized_datasets

## Step 4: Load Pre-trained Model
We will use a pre-trained `DistilBERT` model for token classification.

In [ ]:
# Load a pre-trained model for token classification
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

## Step 5: Training
We will train the model using the `Trainer` API.

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define the metric
metric = load_metric("seqeval", trust_remote_code=True)

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2) # get the argmax of predictions # begin code() # end code()

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    #create list of actual labels
    true_labels = [ # create list of actual labels # begin code() # end code()
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

## Step 6: Evaluation
We will evaluate the model on the test set.

In [ ]:
### Ex-5-Task-1
f1_score, precision, recall = None, None, None

### BEGIN SOLUTION
# Replace the resulted metric obtained from training
# Put the metric resulted in the last epoch of the training
raise NotImplementedError()
### END SOLUTION

In [ ]:
# Evaluate the model
results = trainer.evaluate(tokenized_datasets["test"])


# Print the evaluation results
print(results)

In [ ]:
# Function for NER inference
def ner_inference(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device) # tokenize the texts and load it in cuda # begin code() # end code()
    with torch.no_grad():
        outputs = model(**inputs) # pass input to model # begin code() # end code()
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)
    results = []
    for i, text in enumerate(texts):
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][i]) # convert ids to token # begin code() # end code()
        pred = predictions[i].tolist()
        result = [(token, label_list[p]) for token, p in zip(tokens, pred) if token not in ["[CLS]", "[SEP]", "[PAD]"]]
        results.append(result)
    return results

In [ ]:
# Example sentences for NER inference
texts = ["Hugging Face Inc. is a company based in New York City.", "The quick brown fox jumps over the lazy dog."]
ner_results = ner_inference(texts)

# Print inference results
for i, result in enumerate(ner_results):
    print(f"Sentence {i+1}:")
    for token, label in result:
        print(f"{token}: {label}")